In [14]:
from heapq import heappop, heappush
import numpy as np
from numpy import cos, pi
import scipy.signal
import soundfile as sf
import cv2
import math
import matplotlib.pyplot as plt
import sounddevice as sd
from scipy.io.wavfile import write, read
from scipy.signal import correlate

# Grabación

In [15]:
fs = 48000

seconds = 177 # Duración grabación
#t = np.linspace(0,seconds,int(seconds*fs)) cambiarle el nombre

In [16]:
#myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
#sd.wait()  # esperamos hasta que el audio es grabado

In [17]:
#write('señalRecibida.wav', fs, myrecording)  # guardar como archivo WAV

# Detección del sincronía

In [18]:
t_win = 0.5 #[s]
t_sol = 0.5 #[s]
ns_win = int(t_win*fs)
ns_sol = int(t_sol*fs)

In [19]:
#audio = read("señalRecibida.wav")
audio = read("mod.wav")
data = audio[1]

In [20]:
T=np.arange(0, len(data)/fs, t_sol)

In [21]:
corr=[]
for n_window in range(math.floor((len(data))/(ns_sol))):
    inf = ns_sol*n_window
    sup = inf+ns_win
    y_w = data[inf:sup]
    Ts = np.arange(0, len(y_w))
    sig_ref=np.cos(2*np.pi*5000*Ts)
    cor = correlate(sig_ref, y_w, mode='full', method='fft')
    corr.append(sum(cor**2))

# Demodulación

In [22]:
# portadoras de canales de imagen
fmin=10000
fmax=20000
[fch1_0, fch2_0, fch3_0, fch1_1, fch2_1, fch3_1] = np.linspace(fmin, fmax, 6, dtype=int)
vec_f = [fch1_0, fch2_0, fch3_0, fch1_1, fch2_1, fch3_1] 

# portadoras de canal de texto
f0t = 3000
f1t = 4000

In [23]:
# periodos para cada simbolo
Np = 6
Nv = int(Np/(min(vec_f)/fs))+1

NpT = 300
NvT = int(NpT/(f0t/fs))+1

In [24]:
# Parámetros generados en emisor, necesarios para el receptor
largos = [439, 285462, 233826, 203653]
a = [150, 40, 35]
ranges = [[10,251], [99, 167], [104, 188]]

In [25]:
# Archivos generados en emisor, necesarios para el receptor
path='C://Users//lesli//My Drive//La U//8vo semestre//EL4112 - Principios de Comunicación//Proyecto 2//'
dic_cht = np.load(path+'dic_cht.npy', allow_pickle = True)
dic_ch1 = np.load(path+'dic_ch1.npy', allow_pickle = True)
dic_ch2 = np.load(path+'dic_ch2.npy', allow_pickle = True)
dic_ch3 = np.load(path+'dic_ch3.npy', allow_pickle = True)
data_cht = np.load(path+'data_cht.npy', allow_pickle = True)
data_ch1 = np.load(path+'data_ch1.npy', allow_pickle = True)
data_ch2 = np.load(path+'data_ch2.npy', allow_pickle = True)
data_ch3 = np.load(path+'data_ch3.npy', allow_pickle = True)

In [26]:
data_sinc = data[:3*fs]
data_info = data[3*fs:]

In [27]:
t = np.arange(0, len(data_info)/fs, 1/fs) #s

In [28]:
CT = np.zeros((2, largos[0]))
C_ch1 = np.zeros((2, largos[1]))
C_ch2 = np.zeros((2, largos[2]))
C_ch3 = np.zeros((2, largos[3]))

#texto
for i in range(largos[0]):
    t_w = t[i*NvT:(i+1)*NvT]
    data_w = data_info[i*NvT:(i+1)*NvT]
    ref0 = np.cos(2*np.pi*f0t*t_w)
    ref1 = np.cos(2*np.pi*f1t*t_w)
    CT[0][i] = sum(scipy.signal.correlate(data_w, ref0, mode='same', method='fft')**2)
    CT[1][i] = sum(scipy.signal.correlate(data_w, ref1, mode='same', method='fft')**2)

In [29]:
#ch1
for i in range(largos[1]):
    t_w = t[i*Nv:(i+1)*Nv]
    data_w = data_info[i*Nv:(i+1)*Nv]
    ref0 = np.cos(2*np.pi*fch1_0*t_w)
    ref1 = np.cos(2*np.pi*fch1_1*t_w)
    C_ch1[0][i] = sum(scipy.signal.correlate(data_w, ref0, mode='same', method='fft')**2)
    C_ch1[1][i] = sum(scipy.signal.correlate(data_w, ref1, mode='same', method='fft')**2)

In [30]:
#ch2
for i in range(largos[2]):
    t_w = t[i*Nv:(i+1)*Nv]
    data_w = data_info[i*Nv:(i+1)*Nv]
    ref0 = np.cos(2*np.pi*fch2_0*t_w)
    ref1 = np.cos(2*np.pi*fch2_1*t_w)
    C_ch2[0][i] = sum(scipy.signal.correlate(data_w, ref0, mode='same', method='fft')**2)
    C_ch2[1][i] = sum(scipy.signal.correlate(data_w, ref1, mode='same', method='fft')**2)

In [31]:
#ch3
for i in range(largos[3]):
    t_w = t[i*Nv:(i+1)*Nv]
    data_w = data_info[i*Nv:(i+1)*Nv]
    ref0 = np.cos(2*np.pi*fch3_0*t_w)
    ref1 = np.cos(2*np.pi*fch3_1*t_w)
    C_ch3[0][i] = sum(scipy.signal.correlate(data_w, ref0, mode='same', method='fft')**2)
    C_ch3[1][i] = sum(scipy.signal.correlate(data_w, ref1, mode='same', method='fft')**2)

In [32]:
def demodular(largo, C0, C1, data_ch, sh_error = False):
    demod = np.zeros(largo)
    for i in range(largo):
        if C0[i] > C1[i]:
            demod[i] = 0
        else:
            demod[i] = 1
    if sh_error:
        error = demod-data_ch
        error = sum(abs(error))
        print('Error:', error)
    return demod.astype(int)

demod_cht = demodular(largos[0], CT[0], CT[1], data_cht, True)
demod_ch1 = demodular(largos[1], C_ch1[0], C_ch1[1], data_ch1, True)
demod_ch2 = demodular(largos[2], C_ch2[0], C_ch2[1], data_ch2, True)
demod_ch3 = demodular(largos[3], C_ch3[0], C_ch3[1], data_ch3, True)

Error: 208.0
Error: 0.0
Error: 0.0
Error: 0.0


# Decodificación

In [33]:
# dado un valor obtiene su llave en un diccionario
def obtenerLlave(valor,dic):
    for llaves,valores in dic.items():
        if valores==valor:
            return llaves

In [34]:
def decod_text(demod, dic):
    decod=''
    i=0
    j=0
    k=0    
    while i<len(demod):
        code_i=demod[j:i+1]
        if list(code_i) in dic.values():
            llave=obtenerLlave(list(code_i), dic)
            decod+=llave
            i+=1
            j=i
            k+=1
        else:
            i+=1
    return decod

In [35]:
def decod(demod, dic):
    decod=np.zeros(225**2, dtype=int)
    i=0
    j=0
    k=0    
    while i<len(demod):
        code_i=demod[j:i+1]
        if list(code_i) in dic.values():
            llave=obtenerLlave(list(code_i), dic)
            decod[k]=llave
            i+=1
            j=i
            k+=1
        else:
            i+=1
    return decod

In [36]:
def numpy_dic(dic):
    dic_aux = []
    for i in range(len(dic)):
        dic_aux.append([dic[i][0], dic[i][1]])
    return dict(dic_aux)

In [37]:
# Reestructuración diccionario
dic_cht=numpy_dic(dic_cht)
dic_ch1=numpy_dic(dic_ch1)
dic_ch2=numpy_dic(dic_ch2)
dic_ch3=numpy_dic(dic_ch3)

# Texto

In [38]:
decod_cht=decod_text(demod_cht, dic_cht)
decod_cht

'4tur1oñ22o0Nión clitustLaO2n doMl2danioe Sd 2n e1tptcpcc yg0,oouuoirca-aLPm!eoec2cFtsCerl'

# Imagen

In [39]:
decod_ch1=decod(demod_ch1, dic_ch1).reshape(225, 225).astype('uint8')
decod_ch2=decod(demod_ch2, dic_ch2).reshape(225, 225).astype('uint8')
decod_ch3=decod(demod_ch3, dic_ch3).reshape(225, 225).astype('uint8')

In [40]:
def contrastStre(ch, a, ranges):
    new_ch = np.zeros(np.shape(ch), dtype=int)
    min= ranges[0]
    max = ranges[1]
    for i in range(ch.shape[0]):
        for j in range(ch.shape[1]):
            new_ch[i,j] = int(ch[i,j]/a*(max-min)+min)

    return new_ch.clip(0,255)

In [41]:
new_decod_ch1 = contrastStre(decod_ch1, a[0], ranges[0])
new_decod_ch2 = contrastStre(decod_ch2, a[1], ranges[1])
new_decod_ch3 = contrastStre(decod_ch3, a[2], ranges[2])
Im = cv2.merge([new_decod_ch1, new_decod_ch2, new_decod_ch3])

In [44]:
cv2.imwrite('yuv_receptor.png', Im)

img_yuv = cv2.imread('yuv_receptor.png')
img_rgb = cv2.cvtColor(Im, cv2.COLOR_YUV2RGB)
cv2.imshow('receptor', img_rgb)
cv2.imwrite('rgb_receptor.png', img_rgb)
cv2.waitKey(0) 
cv2.destroyAllWindows() 